In [ ]:
from rosemary import jpt_parse_args, jpt_setup; jpt_setup()

import platform
import sys
sys.path.append('/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/'
                if platform.uname().processor == 'x86_64' 
                else '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import argparse
import os
import torch
import numpy as np
import pandas as pd
import time
import json
from tqdm import tqdm
import time
from eval.mmlu.categories import subcategories, categories
from eval.utils import get_next_word_predictions, load_hf_lm_and_tokenizer, query_openai_chat_model
import torch
from transformers import GPT2LMHeadModel

from eval.mmlu.run_eval import format_subject, format_example, gen_prompt, eval_hf_model

choices = ["A", "B", "C", "D"]

In [13]:

parser = argparse.ArgumentParser()
parser.add_argument("--ntrain", type=int, default=5)
parser.add_argument("--data_dir", type=str, default="data/mmlu")
parser.add_argument("--save_dir", type=str, default="results/mmlu/llama-7B/")
parser.add_argument("--model_name_or_path", type=str, default=None, help="if specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="if specified, we will load the tokenizer from here.")
parser.add_argument("--openai_engine", type=str, default=None, help="if specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--subjects", nargs="*", help="which subjects to evaluate. If not specified, all the 57 subjects will be evaluated.")
parser.add_argument("--n_instances", type=int, help="if specified, a maximum of n_instances per subject will be used for the evaluation.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="batch size for evaluation.")
parser.add_argument("--load_in_8bit", action="store_true", help="load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, the prompt will be encoded as a chat format with the roles in prompt.")

model_name_or_path = '../results/baselines/huggyllama/llama-7b/'
model_name_or_path = '../results/huggyllama:llama-7b_human_mix-trainer_savebystep/checkpoint-1400'
model_name_or_path = '../results/baselines/gpt2-medium/'

cmd = f"""
    --ntrain 0 \
    --data_dir ../data/eval/mmlu \
    --save_dir {model_name_or_path}/eval/mmlu/ \
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name_or_path {model_name_or_path} \
    --eval_batch_size 40 \
    --use_chat_format
"""
# --eval_batch_size 2 
#     --n_instances 10
args = jpt_parse_args(parser, cmd)
args

Namespace(ntrain=0, data_dir='../data/eval/mmlu', save_dir='../results/baselines/gpt2-medium//eval/mmlu/', model_name_or_path='../results/baselines/gpt2-medium/', tokenizer_name_or_path='../results/baselines/gpt2-medium/', openai_engine=None, subjects=None, n_instances=None, eval_batch_size=40, load_in_8bit=False, gptq=False, use_chat_format=True)

In [4]:

if args.model_name_or_path:
    print("Loading model and tokenizer...")
    model, tokenizer = load_hf_lm_and_tokenizer(
        model_name_or_path=args.model_name_or_path, 
        tokenizer_name_or_path=args.tokenizer_name_or_path,
        load_in_8bit=args.load_in_8bit, 
        gptq_model=args.gptq
    )
model.device, model.dtype

Using pad_token, but it is not set yet.


Loading model and tokenizer...


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at ../results/baselines/gpt2-medium/ and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(device(type='cuda', index=0), torch.float16)

In [14]:

subjects = sorted(
    [
        f.split("_test.csv")[0]
        for f in os.listdir(os.path.join(args.data_dir, "test"))
        if "_test.csv" in f
    ]
)

if args.subjects:
    assert all(subj in subjects for subj in args.subjects), f"Some of the subjects you specified are not valid: {args.subjects}"
    subjects = args.subjects


if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)
if not os.path.exists(os.path.join(args.save_dir)):
    os.makedirs(os.path.join(args.save_dir))
if not os.path.exists(os.path.join(args.save_dir, "csvs")):
    os.makedirs(os.path.join(args.save_dir, "csvs"))
    

# wpq: for gpt-2 model, need to enforce `max_length` constraints to avoid `position_id` index errors.
if isinstance(model, GPT2LMHeadModel):
    max_input_seq_len = model.config.max_position_embeddings-1
else:
    max_input_seq_len = 2048-1

all_cors = []
subcat_cors = {
    subcat: [] for subcat_lists in subcategories.values() for subcat in subcat_lists
}
cat_cors = {cat: [] for cat in categories}

subjects


['abstract_algebra',
 'anatomy',
 'astronomy',
 'business_ethics',
 'clinical_knowledge',
 'college_biology',
 'college_chemistry',
 'college_computer_science',
 'college_mathematics',
 'college_medicine',
 'college_physics',
 'computer_security',
 'conceptual_physics',
 'econometrics',
 'electrical_engineering',
 'elementary_mathematics',
 'formal_logic',
 'global_facts',
 'high_school_biology',
 'high_school_chemistry',
 'high_school_computer_science',
 'high_school_european_history',
 'high_school_geography',
 'high_school_government_and_politics',
 'high_school_macroeconomics',
 'high_school_mathematics',
 'high_school_microeconomics',
 'high_school_physics',
 'high_school_psychology',
 'high_school_statistics',
 'high_school_us_history',
 'high_school_world_history',
 'human_aging',
 'human_sexuality',
 'international_law',
 'jurisprudence',
 'logical_fallacies',
 'machine_learning',
 'management',
 'marketing',
 'medical_genetics',
 'miscellaneous',
 'moral_disputes',
 'moral_sce

# function `eval_hf_model`

In [15]:
subject = subjects[9]
print(subject)

dev_df = pd.read_csv(
    os.path.join(args.data_dir, "dev", subject + "_dev.csv"), header=None
)[: args.ntrain]
test_df = pd.read_csv(
    os.path.join(args.data_dir, "test", subject + "_test.csv"), header=None
)
if args.n_instances and args.n_instances < test_df.shape[0]:
    test_df = test_df.sample(args.n_instances, random_state=42)

college_medicine


In [70]:


# cors, acc, probs = eval_hf_model(
#     args, subject, model, tokenizer, dev_df, test_df, args.eval_batch_size)
batch_size = args.eval_batch_size

prompts = []
for i in range(0, test_df.shape[0]):
    prompt_end = format_example(test_df, i, include_answer=False)
    for k in list(range(-1, args.ntrain+1)[::-1]):
        ## wpq: in case zero-shot ICL exceeds `max_input_seq_len`
        # truncate the question on the left.
        if k == -1:
            tokenized_prompt_end = tokenizer(prompt_end, return_tensors="pt", add_special_tokens=False).input_ids
            if args.use_chat_format:
                prompt_other_than_prompt_end = "<|user|>\n"+train_prompt+"\n<|assistant|>\nThe answer is:"
            else:
                prompt_other_than_prompt_end = train_prompt
            tokenized_prompt_other_than_prompt_end = tokenizer(prompt_other_than_prompt_end, return_tensors="pt", add_special_tokens=False).input_ids
            train_prompt_max_len = max_input_seq_len-tokenized_prompt_other_than_prompt_end.shape[-1]
            prompt_end = tokenizer.decode(tokenized_prompt_end.squeeze()[-train_prompt_max_len:], skip_special_tokens=False)
            print(f'Truncate question #{i}: seq_len = {tokenized_prompt.shape[-1]} -> {max_input_seq_len}')

        train_prompt = gen_prompt(dev_df, subject, k)
        prompt = train_prompt + prompt_end
        if args.use_chat_format:
            prompt = "<|user|>\n" + prompt.strip() + "\n<|assistant|>\nThe answer is:"

        tokenized_prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids
        if tokenized_prompt.shape[-1] <= max_input_seq_len:
            break
    prompts.append(prompt)
prompts[0]

Truncate question #66: seq_len = 1029 -> 1023


IndexError: list index out of range

In [42]:
train_prompt

'The following are multiple choice questions (with answers) about  college medicine.\n\n'

In [57]:
prompt_start_ctx_and_end = tokenizer(
    "<|user|>\n"+train_prompt+"\n<|assistant|>\nThe answer is:", 
    return_tensors="pt", add_special_tokens=False).input_ids
prompt_start_ctx_and_end.shape

prompt_start_ctx_and_end.shape[-1]

34

In [62]:
k = 0
prompt_end = format_example(test_df, 66, include_answer=False)

train_prompt = gen_prompt(dev_df, subject, k)
prompt = train_prompt + prompt_end
if args.use_chat_format:
    prompt = "<|user|>\n" + prompt.strip() + "\n<|assistant|>\nThe answer is:"

tokenized_prompt = tokenizer(prompt_end, return_tensors="pt", add_special_tokens=False).input_ids
# tokenizer.decode(tokenized_prompt.squeeze()[], skip_special_tokens=False)



# train_prompt_max_len = max_input_seq_len-prompt_start_ctx_and_end.shape[-1]

# tokenized_prompt[-prompt_start_ctx_and_end:].shape, tokenized_prompt.shape, train_prompt_max_len



[' referred to as "sauna bathing," is characterized by short-term passive exposure to extreme heat. This exposure elicits mild hyperthermia – an increase in the body\'s core temperature – that induces a thermoregulatory response involving neuroendocrine, cardiovascular, and cytoprotective mechanisms that work together to restore homeostasis and condition the body for future heat stressors… In recent decades, sauna bathing has emerged as a means to increase lifespan and improve overall health, based on compelling data from observational, interventional, and mechanistic studies. Of particular interest are the findings from studies of participants in the Kuopio Ischemic Heart Disease Risk Factor (KIHD) Study, an ongoing prospective population-based cohort study of health outcomes in more than 2,300 middle-aged men from eastern Finland, which identified strong links between sauna use and reduced death and disease… The KIHD findings showed that men who used the sauna two to three times per 

In [24]:
prompt_end

'The descending loop of Henle of the nephron of the kidney is permeable to which of the following substances?\nA. Na+\nB. H2O\nC. K+\nD. Cl-\nAnswer:'

In [22]:
x = tokenizer(prompts, padding=True, return_tensors='pt')
x.input_ids.shape


torch.Size([173, 1029])

In [8]:
x = tokenizer(prompts, padding=True, return_tensors='pt')
input_ids = x.input_ids
input_ids = input_ids[:3].cuda()
attention_mask = x.attention_mask
attention_mask = attention_mask[:3].cuda()

batch_logits = model(input_ids=input_ids,
                     attention_mask=attention_mask).logits[:, -1, :]
batch_logits




tensor([[-47.2500, -47.6250, -51.3125,  ..., -54.5000, -55.4062, -46.1250],
        [-42.4062, -43.5938, -46.7812,  ..., -49.7188, -51.4688, -41.0312],
        [-42.4688, -43.5000, -46.6562,  ..., -49.6250, -51.5000, -41.2812]],
       device='cuda:0', dtype=torch.float16, grad_fn=<SliceBackward0>)

In [9]:
input_ids.shape

torch.Size([3, 1011])

In [ ]:

# get the answer for all examples
# note: here we cannot directly use convert_tokens_to_ids because the some tokenizers will automatically add space prefix.
answer_choice_ids = [tokenizer.encode(answer_choice, add_special_tokens=False)[0] for answer_choice in choices]
pred_indices, all_probs = get_next_word_predictions(
    model, tokenizer, prompts, candidate_token_ids=answer_choice_ids, return_token_predictions=False, batch_size=batch_size
)

# get the metrics
cors = []
groud_truths = test_df.iloc[:, -1].values
for i in range(len(pred_indices)):
    prediction = choices[pred_indices[i]]
    ground_truth = groud_truths[i]
    cors.append(prediction == ground_truth)

acc = np.mean(cors)
cors = np.array(cors)

all_probs = np.array(all_probs)
print("Average accuracy {:.3f} - {}".format(acc, subject))

# main()

In [ ]:
l = []
for subject in subjects:
    test_df = pd.read_csv(
        os.path.join(args.data_dir, "test", subject + "_test.csv"), header=None
    )
    l.append((subject, len(test_df)))
    
l.append(('avg', sum([x[1] for x in l])))
l # 14k problems.

In [ ]:


for subject in tqdm(subjects, desc=f"Evaluating subjects: "):

    dev_df = pd.read_csv(
        os.path.join(args.data_dir, "dev", subject + "_dev.csv"), header=None
    )[: args.ntrain]
    test_df = pd.read_csv(
        os.path.join(args.data_dir, "test", subject + "_test.csv"), header=None
    )
    if args.n_instances and args.n_instances < test_df.shape[0]:
        test_df = test_df.sample(args.n_instances, random_state=42)

    if args.model_name_or_path:
        cors, acc, probs = eval_hf_model(args, subject, model, tokenizer, dev_df, test_df, args.eval_batch_size)
    else:
        cors, acc, probs = eval_openai_chat_engine(args, subject, args.openai_engine, dev_df, test_df, args.eval_batch_size)

    subcats = subcategories[subject]
    for subcat in subcats:
        subcat_cors[subcat].append(cors)
        for key in categories.keys():
            if subcat in categories[key]:
                cat_cors[key].append(cors)
    all_cors.append(cors)

    test_df["correct"] = cors
    for j in range(probs.shape[1]):
        choice = choices[j]
        test_df["choice{}_probs".format(choice)] = probs[:, j]
    test_df.to_csv(
        os.path.join(
            args.save_dir, 'csvs', "{}.csv".format(subject)
        ),
        index=None,
    )

for subcat in subcat_cors:
    subcat_acc = np.mean(np.concatenate(subcat_cors[subcat]))
    print("Average accuracy {:.3f} - {}".format(subcat_acc, subcat))

for cat in cat_cors:
    cat_acc = np.mean(np.concatenate(cat_cors[cat]))
    print("Average accuracy {:.3f} - {}".format(cat_acc, cat))
weighted_acc = np.mean(np.concatenate(all_cors))
print("Average accuracy: {:.3f}".format(weighted_acc))

# save results
with open(os.path.join(args.save_dir, "metrics.json"), "w") as f:
    json.dump(
        {
            "average_acc": weighted_acc,
            "subcat_acc": {
                subcat: np.mean(np.concatenate(subcat_cors[subcat]))
                for subcat in subcat_cors
            },
            "cat_acc": {
                cat: np.mean(np.concatenate(cat_cors[cat]))
                for cat in cat_cors
            },
        },
        f,
    )

